## Load dataset

In [13]:
import glob
import pandas as pd

import numpy as np

def highlight_max(data):
    percent = data.str.contains('%').any()

    data = data.str.split(expand=True)
    percent = data[0].str.contains('%').any()

    data[0] = data[0].replace('\\\%','', regex=True).replace('\$','', regex=True).astype(float)
    data[0] = np.where(data[0] == data[0].max(), '\mathbf{'+ str(data[0].max()) + '}', data[0])
    if percent:
        return '$' + data[0] + '\% ' + data[1] + ' ' + data[2]
    else:
        return '$' + data[0] + ' ' + data[1] + ' ' + data[2]

    return data

dataset_path = 'full'
dataset_path = 'amauri'
dataset_path = 'full_with_tfidf'
#dataset_path = 'small'

files = glob.glob(f'../evaluate_results/{dataset_path}/outer/*.csv', recursive=True)

print(len(files), 'files')
files[0:3]

20 files


["../evaluate_results/full_with_tfidf/outer/KNeighborsClassifier-split_x_y-{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60, 80, 100], 'algorithm': ['brute'], 'metric': ['hamming']}-accuracy-(2 of 5).csv",
 "../evaluate_results/full_with_tfidf/outer/SVC-split_x_y-{'C': [0.002, 2.0, 2000.0, 2000000.0, 2000000000.0, 2000000000000.0], 'gamma': [2e-13, 2e-10, 2e-07, 0.0002, 0.2, 200.0], 'kernel': ['rbf'], 'probability': [True]}-accuracy-(4 of 5).csv",
 "../evaluate_results/full_with_tfidf/outer/SVC-split_x_y-{'C': [0.002, 2.0, 2000.0, 2000000.0, 2000000000.0, 2000000000000.0], 'gamma': [2e-13, 2e-10, 2e-07, 0.0002, 0.2, 200.0], 'kernel': ['rbf'], 'probability': [True]}-accuracy-(5 of 5).csv"]

In [14]:
# Read all data
data = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])
data.head(2)

,best_params,column,i_outer,metric,model,params,refit,split_method,value
0,"{'algorithm': 'brute', 'metric': 'hamming', 'n...",0,1,accuracy,KNeighborsClassifier,"{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60,...",accuracy,split_x_y,0.446759
1,"{'algorithm': 'brute', 'metric': 'hamming', 'n...",0,1,hit@5,KNeighborsClassifier,"{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60,...",accuracy,split_x_y,0.604167


## Results

In [15]:
result = data.groupby(['model', 'split_method', 'metric'])['value'].agg(["mean", "std"])
result.head(3)

mean       std
model                split_method metric                      
KNeighborsClassifier split_x_y    accuracy  0.342662  0.135818
                                  hit@5     0.520976  0.131489
                                  map@5     0.300250  0.156469

## Result as pivot table

(Beauty table bellow)

In [16]:
pivot_table = result.pivot_table(index=['model', 'split_method'], columns='metric', values=['mean', 'std'])
pivot_table

mean                                \
metric                             accuracy     hit@5     map@5      mdcg   
model                split_method                                           
KNeighborsClassifier split_x_y     0.342662  0.520976  0.300250  0.539627   
LogisticRegression   split_x_y     0.349445  0.548577  0.304107  0.555884   
MLPClassifier        split_x_y     0.328927  0.538246  0.303420  0.544141   
SVC                  split_x_y     0.244419  0.479110  0.295142  0.492183   

                                        std                                
metric                             accuracy     hit@5     map@5      mdcg  
model                split_method                                          
KNeighborsClassifier split_x_y     0.135818  0.131489  0.156469  0.107570  
LogisticRegression   split_x_y     0.134689  0.127521  0.155862  0.105395  
MLPClassifier        split_x_y     0.136555  0.130974  0.153501  0.107545  
SVC                  split_x_y     0.153681  0.167067  0.174017  0.127315

### 'Cute' Pivot table

In [17]:
data = pd.read_csv('../rbm_results.csv', index_col=0)

data['metric'].replace('Accuracy', 'accuracy', inplace=True)
data['metric'].replace('Hit@5', 'hit@5', inplace=True)
data['metric'].replace('MAP@5', 'map@5', inplace=True)
data['metric'].replace('MDCG', 'mdcg', inplace=True)


data['total_missing'] = data['missing'].str.split(',').str.len() + 1
data['total_missing'] = np.where(data['missing'] == 'set()', 1, data['total_missing'])
rbm_pivot = data.groupby(['metric', 'total_missing'])['value'].agg(["mean", "std"]) \
    .pivot_table(index=['total_missing'], columns='metric', values=['mean', 'std'])
rbm_pivot.iloc[0]
rbm_hide_1 = rbm_pivot.iloc[0].to_frame().T
rbm_hide_1.index = [('RBM', 'split_x_y_split_with_one_hot_encoding')]
rbm_hide_1

mean                     \
metric                                        accuracy    hit@5     map@5   
(RBM, split_x_y_split_with_one_hot_encoding)  0.306567  0.46382  0.274736   

                                                             std            \
metric                                            mdcg  accuracy     hit@5   
(RBM, split_x_y_split_with_one_hot_encoding)  0.503321  0.136337  0.133142   

                                                                  
metric                                           map@5      mdcg  
(RBM, split_x_y_split_with_one_hot_encoding)  0.147204  0.108026

In [18]:
def format_percentage(x):
    return '{:2.2%}'.format(x).replace('%', '\%')

def format_std(x):
    return '{:.4f}'.format(x)

pivot_table2 = pd.concat([pivot_table, rbm_hide_1])

new_pivot_table = pd.DataFrame({
    'Accuracy (std)': '$' + pivot_table2['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table2['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5 (std)':    '$' + pivot_table2['mean']['hit@5'].map(format_percentage)    + ' \pm ' + pivot_table2['std']['hit@5'].map(format_percentage) + "$",
    'MDCG (std)':     '$' + pivot_table2['mean']['mdcg'].map(format_std)            + ' \pm ' + pivot_table2['std']['mdcg'].map(format_std) + "$",
    'MAP@5 (std)':    '$' + pivot_table2['mean']['map@5'].map(format_percentage)    + ' \pm ' + pivot_table2['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

new_pivot_table.apply(highlight_max).to_latex("table.tex", escape=False)
new_pivot_table.apply(highlight_max)


,,Accuracy (std),Hit@5 (std),MDCG (std),MAP@5 (std)
Model,Embedding,,,,
$k$-NN,No embedding,$34.27\% \pm 13.58\%$,$52.1\% \pm 13.15\%$,$0.5396 \pm 0.1076$,$30.03\% \pm 15.65\%$
Logistic Regression,No embedding,$\mathbf{34.94}\% \pm 13.47\%$,$\mathbf{54.86}\% \pm 12.75\%$,$\mathbf{0.5559} \pm 0.1054$,$\mathbf{30.41}\% \pm 15.59\%$
MLP,No embedding,$32.89\% \pm 13.66\%$,$53.82\% \pm 13.10\%$,$0.5441 \pm 0.1075$,$30.34\% \pm 15.35\%$
SVC,No embedding,$24.44\% \pm 15.37\%$,$47.91\% \pm 16.71\%$,$0.4922 \pm 0.1273$,$29.51\% \pm 17.40\%$
RBM,One-hot concatenated,$30.66\% \pm 13.63\%$,$46.38\% \pm 13.31\%$,$0.5033 \pm 0.1080$,$27.47\% \pm 14.72\%$


###### Load dataset (hide two columns)

### Two hidden columns

In [62]:
files = glob.glob(f'../evaluate_results/hide_two/outer/*.csv', recursive=True)

# Read all data
data_hide_two = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])

result = data_hide_two.groupby(['model', 'split_method', 'metric', 'column'])['value'].agg(["mean", "std"])
pivot_table = result.pivot_table(index=['model', 'split_method', 'column'], columns='metric', values=['mean', 'std'])


new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

print("Duas colunas são ocultadas. A indicada em 'Column' é o índice da coluna colocada como 'y'. A outra coluna ocultada segue a seguinte regra")
print("Obs: A 5º coluna é mostrada como '3' pq o índice começa como zero e pq eu tiro a 2º coluna para computar os dados")
print("  - if column = 1, then the other hidden column is 5: P(column_1 | columns \ {column_1, column_5})")
print("  - if column = 5, then the other hidden column is 1: P(column_5 | columns \ {column_1, column_5})")
new_pivot_table.apply(highlight_max)

Duas colunas são ocultadas. A indicada em 'Column' é o índice da coluna colocada como 'y'. A outra coluna ocultada segue a seguinte regra
Obs: A 5º coluna é mostrada como '3' pq o índice começa como zero e pq eu tiro a 2º coluna para computar os dados
  - if column = 1, then the other hidden column is 5: P(column_1 | columns \ {column_1, column_5})
  - if column = 5, then the other hidden column is 1: P(column_5 | columns \ {column_1, column_5})


Accuracy  \
Model               Embedding            Column                                 
$k$-NN              No embedding         1               $20.18\% \pm 1.35\%$   
                                         3               $44.05\% \pm 2.42\%$   
Logistic Regression One-hot concatenated 1               $24.06\% \pm 2.25\%$   
                                         3               $46.65\% \pm 1.67\%$   
MLP                 One-hot concatenated 1               $20.92\% \pm 1.79\%$   
                                         3               $42.71\% \pm 0.89\%$   
SVC                 One-hot concatenated 1               $25.13\% \pm 1.95\%$   
                                         3       $\mathbf{47.2}\% \pm 1.46\%$   

                                                                         Hit@5  \
Model               Embedding            Column                                  
$k$-NN              No embedding         1                $39.75\% \pm 2.07\%$   
                                         3                $58.91\% \pm 2.03\%$   
Logistic Regression One-hot concatenated 1                $47.39\% \pm 1.18\%$   
                                         3       $\mathbf{66.87}\% \pm 0.92\%$   
MLP                 One-hot concatenated 1                 $43.5\% \pm 1.85\%$   
                                         3                $62.29\% \pm 1.81\%$   
SVC                 One-hot concatenated 1                $46.83\% \pm 1.89\%$   
                                         3                $66.45\% \pm 1.12\%$   

                                                                         MDCG  \
Model               Embedding            Column                                 
$k$-NN              No embedding         1                $0.4291 \pm 0.0177$   
                                         3                $0.6037 \pm 0.0177$   
Logistic Regression One-hot concatenated 1                $0.4815 \pm 0.0166$   
                                         3       $\mathbf{0.6516} \pm 0.0104$   
MLP                 One-hot concatenated 1                $0.4508 \pm 0.0153$   
                                         3                $0.6182 \pm 0.0034$   
SVC                 One-hot concatenated 1                $0.4769 \pm 0.0135$   
                                         3                $0.6509 \pm 0.0095$   

                                                                         MAP@5  
Model               Embedding            Column                                 
$k$-NN              No embedding         1                $14.59\% \pm 0.54\%$  
                                         3                $42.51\% \pm 1.45\%$  
Logistic Regression One-hot concatenated 1                $16.05\% \pm 0.51\%$  
                                         3                 $44.3\% \pm 1.33\%$  
MLP                 One-hot concatenated 1                $15.47\% \pm 0.13\%$  
                                         3                $42.27\% \pm 1.33\%$  
SVC                 One-hot concatenated 1                 $16.4\% \pm 0.75\%$  
                                         3       $\mathbf{44.48}\% \pm 1.22\%$

### Compare with one hidden column

In [7]:
pivot_table = data[
        (data.split_method == 'split_x_y_split_with_one_hot_encoding') 
      & ((data.column == 1) | (data.column == 4))] \
    .groupby(['model', 'split_method', 'metric', 'column'])['value'] \
    .agg(["mean", "std"]) \
    .pivot_table(index=['model', 'split_method', 'column'], columns='metric', values=['mean', 'std'])



new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

print("Somente a coluna indicada está oculta")
print("A 5º coluna é 4 pq não deletei a 2º coluna")
print("  - P(column_n | columns \ {column_n})")

new_pivot_table

Somente a coluna indicada está oculta
A 5º coluna é 4 pq não deletei a 2º coluna
  - P(column_n | columns \ {column_n})


Accuracy  \
Model               Embedding            Column                         
$k$-NN              One-hot concatenated 1       $20.50\% \pm 1.29\%$   
                                         4       $44.01\% \pm 2.21\%$   
Logistic Regression One-hot concatenated 1       $24.20\% \pm 1.11\%$   
                                         4       $47.57\% \pm 0.99\%$   
MLP                 One-hot concatenated 1       $21.19\% \pm 2.72\%$   
                                         4       $42.20\% \pm 2.04\%$   
SVC                 One-hot concatenated 1       $25.40\% \pm 1.05\%$   
                                         4       $47.06\% \pm 1.38\%$   

                                                                Hit@5  \
Model               Embedding            Column                         
$k$-NN              One-hot concatenated 1       $39.43\% \pm 3.04\%$   
                                         4       $60.16\% \pm 2.26\%$   
Logistic Regression One-hot concatenated 1       $47.71\% \pm 1.60\%$   
                                         4       $66.96\% \pm 1.22\%$   
MLP                 One-hot concatenated 1       $43.87\% \pm 2.87\%$   
                                         4       $62.29\% \pm 1.19\%$   
SVC                 One-hot concatenated 1       $45.03\% \pm 2.12\%$   
                                         4       $66.68\% \pm 1.48\%$   

                                                                MDCG  \
Model               Embedding            Column                        
$k$-NN              One-hot concatenated 1       $0.4272 \pm 0.0197$   
                                         4       $0.6111 \pm 0.0118$   
Logistic Regression One-hot concatenated 1       $0.4834 \pm 0.0125$   
                                         4       $0.6579 \pm 0.0068$   
MLP                 One-hot concatenated 1       $0.4537 \pm 0.0181$   
                                         4       $0.6160 \pm 0.0125$   
SVC                 One-hot concatenated 1       $0.4703 \pm 0.0139$   
                                         4       $0.6518 \pm 0.0118$   

                                                                MAP@5  
Model               Embedding            Column                        
$k$-NN              One-hot concatenated 1       $14.50\% \pm 0.37\%$  
                                         4       $43.22\% \pm 1.34\%$  
Logistic Regression One-hot concatenated 1       $16.06\% \pm 0.33\%$  
                                         4       $44.39\% \pm 1.39\%$  
MLP                 One-hot concatenated 1       $15.62\% \pm 0.38\%$  
                                         4       $42.48\% \pm 1.52\%$  
SVC                 One-hot concatenated 1       $16.35\% \pm 0.68\%$  
                                         4       $44.60\% \pm 1.11\%$

Ao comparar as duas tabelas, se conclui que 

* remover a coluna 4 tira pouca informação na previsão da coluna 1;
* remover a coluna 1 tira pouca informação na previsão da coluna 4.

# First two columns

In [8]:
files = glob.glob(f'../evaluate_results/hide_two_hard_p1/outer/*.csv', recursive=True)
files2 = glob.glob(f'../evaluate_results/hide_two_hard/outer/*.csv', recursive=True)

# Read all data
data_hide_two = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])
data_hide_two['column_y'] = 1
data_hide_two['column_ignored'] = 0
data_hide_two2 = pd.concat([pd.read_csv(file, index_col=[0]) for file in files2])
data_hide_two2['column_y'] = 0
data_hide_two2['column_ignored'] = 1

data_hide_two = pd.concat((data_hide_two, data_hide_two2))

result = data_hide_two.groupby(['model', 'split_method', 'metric', 'column_y', 'column_ignored'])['value'].agg(["mean", "std"])
pivot_table = result.pivot_table(index=['model', 'split_method', 'column_y', 'column_ignored'], columns='metric', values=['mean', 'std'])


new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column y', 'Column ignored'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

new_pivot_table

Accuracy  \
Model               Embedding            Column y Column ignored                         
$k$-NN              One-hot concatenated 0        1               $38.41\% \pm 2.74\%$   
                                         1        0               $16.34\% \pm 1.45\%$   
Logistic Regression One-hot concatenated 0        1               $40.49\% \pm 2.97\%$   
                                         1        0               $19.02\% \pm 1.16\%$   
MLP                 One-hot concatenated 0        1               $34.75\% \pm 3.58\%$   
                                         1        0               $16.80\% \pm 0.95\%$   
SVC                 One-hot concatenated 0        1               $41.55\% \pm 2.58\%$   
                                         1        0               $19.67\% \pm 1.10\%$   

                                                                                 Hit@5  \
Model               Embedding            Column y Column ignored                         
$k$-NN              One-hot concatenated 0        1               $56.31\% \pm 3.50\%$   
                                         1        0               $35.63\% \pm 2.66\%$   
Logistic Regression One-hot concatenated 0        1               $61.91\% \pm 2.53\%$   
                                         1        0               $42.71\% \pm 1.87\%$   
MLP                 One-hot concatenated 0        1               $56.55\% \pm 2.25\%$   
                                         1        0               $39.01\% \pm 4.15\%$   
SVC                 One-hot concatenated 0        1               $60.29\% \pm 2.60\%$   
                                         1        0               $40.40\% \pm 2.05\%$   

                                                                                 MDCG  \
Model               Embedding            Column y Column ignored                        
$k$-NN              One-hot concatenated 0        1               $0.5731 \pm 0.0278$   
                                         1        0               $0.3931 \pm 0.0161$   
Logistic Regression One-hot concatenated 0        1               $0.6043 \pm 0.0210$   
                                         1        0               $0.4420 \pm 0.0118$   
MLP                 One-hot concatenated 0        1               $0.5610 \pm 0.0247$   
                                         1        0               $0.4180 \pm 0.0160$   
SVC                 One-hot concatenated 0        1               $0.6077 \pm 0.0182$   
                                         1        0               $0.4326 \pm 0.0142$   

                                                                                 MAP@5  
Model               Embedding            Column y Column ignored                        
$k$-NN              One-hot concatenated 0        1               $28.33\% \pm 1.06\%$  
                                         1        0               $14.27\% \pm 0.45\%$  
Logistic Regression One-hot concatenated 0        1               $33.16\% \pm 0.62\%$  
                                         1        0               $16.41\% \pm 0.46\%$  
MLP                 One-hot concatenated 0        1               $27.38\% \pm 0.53\%$  
                                         1        0               $15.07\% \pm 0.39\%$  
SVC                 One-hot concatenated 0        1               $36.34\% \pm 1.53\%$  
                                         1        0               $16.66\% \pm 0.58\%$

In [9]:
pivot_table = data[
        (data.split_method == 'split_x_y_split_with_one_hot_encoding') 
      & ((data.column == 0) | (data.column == 1))] \
    .groupby(['model', 'split_method', 'metric', 'column'])['value'] \
    .agg(["mean", "std"]) \
    .pivot_table(index=['model', 'split_method', 'column'], columns='metric', values=['mean', 'std'])


new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)


new_pivot_table

Accuracy  \
Model               Embedding            Column                         
$k$-NN              One-hot concatenated 0       $42.20\% \pm 2.85\%$   
                                         1       $20.50\% \pm 1.29\%$   
Logistic Regression One-hot concatenated 0       $45.67\% \pm 2.41\%$   
                                         1       $24.20\% \pm 1.11\%$   
MLP                 One-hot concatenated 0       $42.06\% \pm 3.81\%$   
                                         1       $21.19\% \pm 2.72\%$   
SVC                 One-hot concatenated 0       $46.27\% \pm 1.96\%$   
                                         1       $25.40\% \pm 1.05\%$   

                                                                Hit@5  \
Model               Embedding            Column                         
$k$-NN              One-hot concatenated 0       $57.80\% \pm 2.29\%$   
                                         1       $39.43\% \pm 3.04\%$   
Logistic Regression One-hot concatenated 0       $64.14\% \pm 2.79\%$   
                                         1       $47.71\% \pm 1.60\%$   
MLP                 One-hot concatenated 0       $59.51\% \pm 3.44\%$   
                                         1       $43.87\% \pm 2.87\%$   
SVC                 One-hot concatenated 0       $63.30\% \pm 2.67\%$   
                                         1       $45.03\% \pm 2.12\%$   

                                                                MDCG  \
Model               Embedding            Column                        
$k$-NN              One-hot concatenated 0       $0.5933 \pm 0.0181$   
                                         1       $0.4272 \pm 0.0197$   
Logistic Regression One-hot concatenated 0       $0.6343 \pm 0.0198$   
                                         1       $0.4834 \pm 0.0125$   
MLP                 One-hot concatenated 0       $0.6044 \pm 0.0288$   
                                         1       $0.4537 \pm 0.0181$   
SVC                 One-hot concatenated 0       $0.6333 \pm 0.0157$   
                                         1       $0.4703 \pm 0.0139$   

                                                                MAP@5  
Model               Embedding            Column                        
$k$-NN              One-hot concatenated 0       $25.88\% \pm 0.94\%$  
                                         1       $14.50\% \pm 0.37\%$  
Logistic Regression One-hot concatenated 0       $31.65\% \pm 0.69\%$  
                                         1       $16.06\% \pm 0.33\%$  
MLP                 One-hot concatenated 0       $26.72\% \pm 0.89\%$  
                                         1       $15.62\% \pm 0.38\%$  
SVC                 One-hot concatenated 0       $35.80\% \pm 1.38\%$  
                                         1       $16.35\% \pm 0.68\%$